In [230]:
import glob
import time
import numpy as np
#np.set_printoptions(precision=3)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [231]:
path_base="/home/francisco/datasets/datasets/sound/voice/stress"

In [232]:
def getLabel(s):
    if s=="TRUE":
        return 1.0
    if s=="FALSE":
        return 0.0
    if s=="PC":
        return 2.0
def isRightLabel(s):
    if s=="TRUE":
        return True
    if s=="FALSE":
        return True
    if s=="PC":
        return False
    return False

In [233]:
data=[]
labels=[]
file_txt=path_base+"/*.csv"
for filepath in glob.glob(file_txt):
    print("Processing file: {}".format(filepath)) 
    with open(filepath) as fp:  
        line = fp.readline()
        head_list=line.split(";")
        #for i,head in enumerate(head_list):
        #    print(i,head)
        cnt = 1
        line = fp.readline()
        while line:
            data_row=[]
            line_list=line.split(";")
            #print("Line {}: {} {} {}".format(cnt, line_list[0], line_list[1], line_list[2]))
            line = fp.readline()
            cnt += 1
            if not isRightLabel(line_list[0]):
                continue
            for i,datum in enumerate(line_list):
                #print(i,head_list[i],"=",datum)
                if i>1:
                    data_row.append(float(datum))
                if i==0:
                    labels.append(getLabel(datum))
            data.append(data_row)
data_np=np.array(data)
labels_np=np.array(labels)
print(data_np.shape)
print(data_np)
print(labels_np.shape)
print(labels_np)

Processing file: /home/francisco/datasets/datasets/sound/voice/stress/Set_Males.csv
Processing file: /home/francisco/datasets/datasets/sound/voice/stress/Set_Females.csv
(50, 68)
[[126.278 0.014 0.029 ... 15.681 0.870 0.012]
 [142.901 0.005 0.019 ... 10.834 0.936 0.008]
 [117.189 0.005 0.014 ... 11.765 0.967 0.006]
 ...
 [192.269 0.008 0.022 ... 10.600 0.928 0.007]
 [167.700 0.005 0.025 ... 9.564 0.933 0.009]
 [206.086 0.007 0.013 ... 9.851 0.922 0.046]]
(50,)
[1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 0.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000]


In [234]:
from sklearn.preprocessing import StandardScaler

In [235]:
ss=StandardScaler()
data_ss=ss.fit_transform(data_np)

print(data_ss.shape)
print(data_ss)

(50, 68)
[[-0.692 1.335 0.942 ... 1.056 -1.550 -0.524]
 [-0.309 -0.907 -0.100 ... -0.261 0.011 -0.808]
 [-0.901 -0.907 -0.621 ... -0.008 0.744 -0.950]
 ...
 [0.826 -0.159 0.213 ... -0.325 -0.178 -0.879]
 [0.261 -0.907 0.525 ... -0.606 -0.060 -0.737]
 [1.144 -0.409 -0.725 ... -0.528 -0.320 1.895]]


In [236]:
import numpy as onp
import jax.numpy as np
from jax import grad, jit, vmap, value_and_grad
from jax import random
from jax.experimental import stax
from jax.experimental.stax import (BatchNorm, Conv, Dense, Flatten, Dropout,
                                   Relu, LogSoftmax)

In [237]:
# Import some additional JAX and dataloader helpers
from jax.scipy.special import logsumexp
from jax.experimental import optimizers

In [238]:
#Test if JAX is using CPU or GPU
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [239]:
# Generate key which is used to generate random numbers
key = random.PRNGKey(1)

In [240]:
num_classes = 2
hidden=25
dropout_rate=0.5
def makeNet(num_classes,hidden,dropout_rate,mode="train"):
    init_fun, net = stax.serial(
        Dense(hidden),
        Dropout(dropout_rate,mode=mode),
        Dense(hidden),
        Dropout(dropout_rate,mode=mode),
        Dense(hidden),
        Dense(num_classes),
        LogSoftmax)
    return init_fun,net

In [241]:
init_fun, net=makeNet(num_classes,hidden,dropout_rate)
input_shape=(-1,)+ data_ss.shape[1:]
output_shape, params = init_fun(key, input_shape)
print("ouput_shape=",output_shape)
_,netTest=makeNet(num_classes,hidden,dropout_rate,mode='test')

ouput_shape= (-1, 2)


In [242]:
def one_hot(x, k, dtype=np.float32):
    """Create a one-hot encoding of x of size k """
    return np.array(x[:, None] == np.arange(k), dtype)

def loss(params, data, targets,key):
    preds = net(params, data,rng=key)
    return -np.sum(preds * targets)
def predict(params,data,key):
    preds=netTest(params,data,rng=key)
    return np.exp(preds)

In [243]:
step_size = 1e-3
opt_init, opt_update, get_params = optimizers.adam(step_size)
opt_state = opt_init(params)
num_epochs = 2

In [244]:
@jit
def update(params, x, y, opt_state,key):
    """ Compute the gradient for a batch and update the parameters """
    value, grads = value_and_grad(loss)(params, x, y,key)
    opt_state = opt_update(0, grads, opt_state)
    return get_params(opt_state), opt_state, value

In [245]:
labels_onehot=one_hot(labels_np,num_classes)
print(labels_onehot.shape)
def run_training_loop(data_ss,labels_onehot,num_epochs, opt_state,verbose=False):
    """ Implements a learning loop over epochs. """
    # Get the initial set of parameters
    params = get_params(opt_state)

    # Loop over the training epochs
    for epoch in range(num_epochs):
        start_time = time.time()
        params, opt_state, loss = update(params, data_ss, labels_onehot, opt_state,key)
        epoch_time = time.time() - start_time
        if verbose: print("Epoch {} | T: {:0.2f} | loss: {:0.3f} ".format(epoch+1, epoch_time,
                                                                    loss))
    return loss,params

(50, 2)


In [246]:
l,params=run_training_loop(data_ss,labels_onehot,2000,opt_state)
print(l)

0.0018863175


In [247]:
def _numpy_delete(x, idx):
    """
    Gets the subarray from `x` where data from index `idx` on the first axis is removed.
    """
    # NB: numpy.delete is not yet available in JAX
    mask = np.arange(x.shape[0] - 1) < idx
    return np.where(mask.reshape((-1,) + (1,) * (x.ndim - 1)), x[:-1], x[1:])
def allButOne(x,y,i):
    global key
    oneX=x[i,:]
    oneY=y[i,:]
    allXBut=_numpy_delete(x,i)
    allYBut=_numpy_delete(y,i)
    rng,key=random.split(key)
    idxs=allXBut.shape[0]
    suffleIdx=random.permutation(rng,idxs)
    allXBut=allXBut[suffleIdx]
    allYBut=allYBut[suffleIdx]
    return allXBut,allYBut,oneX,oneY

In [248]:
allXBut,allYBut,oneX,oneY=allButOne(data_ss,labels_onehot,1)
print(allXBut.shape)
print(allYBut.shape)
print(oneX.shape)
print(oneY.shape)
oneY

(49, 68)
(49, 2)
(68,)
(2,)


DeviceArray([0.000, 1.000], dtype=float32)

In [249]:
accT=0
for i in range(data_ss.shape[0]):
    allXBut,allYBut,oneX,oneY=allButOne(data_ss,labels_onehot,i)
    loss,params=run_training_loop(allXBut,allYBut,2000,opt_state)
    p=predict(params,oneX,key)
    acc=np.sum(p*oneY)
    accT+=acc
    print("i=",i,p,oneY,"acc=",acc,"accMean=",accT/(i+1))
print("accMean=",accT/data_ss.shape[0])

i= 0 [0.966 0.034] [0.000 1.000] acc= 0.033999696 accMean= 0.033999696
i= 1 [0.044 0.956] [0.000 1.000] acc= 0.9555793 accMean= 0.49478948
i= 2 [0.780 0.220] [0.000 1.000] acc= 0.22032958 accMean= 0.40330285
i= 3 [0.132 0.868] [0.000 1.000] acc= 0.8684061 accMean= 0.51957864
i= 4 [0.069 0.931] [0.000 1.000] acc= 0.9305687 accMean= 0.60177666
i= 5 [0.785 0.215] [0.000 1.000] acc= 0.21516491 accMean= 0.53734136
i= 6 [0.792 0.208] [0.000 1.000] acc= 0.20816924 accMean= 0.49031678
i= 7 [1.000 0.000] [0.000 1.000] acc= 5.194653e-09 accMean= 0.42902717
i= 8 [0.770 0.230] [0.000 1.000] acc= 0.23034164 accMean= 0.406951
i= 9 [0.009 0.991] [0.000 1.000] acc= 0.9914964 accMean= 0.46540555
i= 10 [0.641 0.359] [1.000 0.000] acc= 0.6405133 accMean= 0.48132446
i= 11 [0.002 0.998] [1.000 0.000] acc= 0.0019398866 accMean= 0.44137573
i= 12 [0.056 0.944] [1.000 0.000] acc= 0.05568412 accMean= 0.41170716
i= 13 [0.647 0.353] [1.000 0.000] acc= 0.64719045 accMean= 0.4285274
i= 14 [0.887 0.113] [1.000 0.000